In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

options = webdriver.ChromeOptions()

browser = webdriver.Chrome(options=options)

url = 'https://www.google.com.tw/maps/place/%E7%93%A6%E5%9F%8E%E6%B3%B0%E5%9C%8B%E6%96%99%E7%90%86+-+%E5%8F%B0%E4%B8%AD%E9%BA%97%E5%AF%B6%E5%BA%97/@24.2335548,120.4303719,10z/data=!4m11!1m2!2m1!1z55Om5Z-O!3m7!1s0x0:0xb4ee93bb18083e72!8m2!3d24.3251213!4d120.7010365!9m1!1b1!15sCgbnk6bln44iA4gBAVoSCgfnk6Yg5Z-OIgfnk6Yg5Z-OkgEPdGhhaV9yZXN0YXVyYW50mgEjQ2haRFNVaE5NRzluUzBWSlEwRm5TVVF3TjJZMmNtWkJFQUU?hl=zh-TW'

browser.implicitly_wait(10)
browser.get(url)

tmp = browser.find_elements_by_xpath("//div[@class='section-review-review-content']")

while len(tmp) < int(611): #評論總數減6
    browser.execute_script('''
        const body = document.querySelector(`
            div.section-layout.section-scrollbox.mapsConsumerUiCommonScrollable__scrollable-y.mapsConsumerUiCommonScrollable__scrollable-show
        `);
        body.scrollTo(0, body.scrollHeight);
    ''')

    time.sleep(0.5)

    tmp = browser.find_elements_by_xpath("//div[@class='section-review-review-content']")

    
soup = BeautifulSoup(browser.page_source, 'html.parser') 
reviewer = soup.find_all('div', {'class': 'section-review-title'})
stars = soup.find_all('span', {'class': 'section-review-stars'})
contents = soup.find_all('span', {'class': 'section-review-text'})

name = []
score = []
review_text = []

for r_name in reviewer:
    name.append(r_name.text)

for rating in stars:
    score.append(rating['aria-label'])
    
for review in contents:
    review_text.append(review.text)

all_data = list(zip(name, score, review_text))

df = pd.DataFrame(all_data, columns = ['評論者' , '評價分數', '評論內容'])

df.to_csv('review.csv', index=False)

df